In [44]:
import pandas as pd
import sqlite3

## 1. Create a connection to the database using the sqlite3 library.

In [45]:
db_path = "../data/checking-logs.sqlite"
conn = sqlite3.connect(db_path)

## 2. Get the schema of the checker table.

In [46]:
checker = pd.read_sql_query("PRAGMA table_info(checker);", conn)
checker

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


## 3. Get only the first ten rows of the table to see what the table looks like.

In [47]:
pageviews = pd.read_sql_query("SELECT * FROM checker LIMIT 10;", conn)
pageviews

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


## 4. Count how many rows satisfy the following conditions using one query with any number of subqueries.

- Count the rows from the pageviews table, but only those from the checker table with:
    - status = 'ready', we do not want to analyze logs that are status checking;
    - numTrials = 1, we only want to analyze the first commits because they tell us when a student started working on a lab;
    - "labnames" should be from the list: laba04, laba04s, laba05, laba06, laba06s, and project1. Only these were active during the experiment.
- Store the checkers in the dataframe with the column "cnt".

In [48]:
cnt = pd.read_sql_query("""
    SELECT COUNT(*) as cnt
    FROM pageviews p
    WHERE EXISTS (
        SELECT 1 
        FROM checker c
        WHERE c.uid = p.uid
        AND c.status = 'ready'
        AND c.numTrials = 1
        AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    )
""", conn)
cnt

,cnt
0,985


# 5. Close the connection.

In [49]:
conn.close()